# Data Frames


In [1]:
using StatsBase, DataFrames, DataFrameMacros
using RCall, CategoricalArrays, TexTables
using Distributions, Gadfly, Compose
using CSV, RDatasets, MLJ
using MLJ: schema

In [2]:
include("pubh.jl");
@rlibrary readr

## Data Manipulation

### Generating Data Frames

For small datasets, the easiest thing to do is to generate the data directly in `julia`. We will start by entering some data of our own. This particular dataset describes the levels of uric acid in the bloodstream of twenty subjects aged from 21 to 25. There were five individuals with each combination of Down's syndrome being present/absent and sex being male/female.

The variable `uric` contains the values for the uric acid, the variable `downs` contains numerical values for Down’s syndrome (0 = "No," 1 = "Yes") and the variable `sex` contains numerical values representing sex (0 = "Male", 1 = "Female").

In [3]:
uric_down = DataFrame(
  uric = [5.84, 6.3, 6.95, 5.92, 7.94, 5.5, 6.08, 5.12, 7.58, 6.78,
  4.9, 6.95, 6.73, 5.32, 4.81, 4.94, 7.2, 5.22, 4.6, 3.88],
  downs = repeat(repeat(0:1, inner=5), 2),
  sex = repeat(0:1, inner=10)
)

uric_down |> head

Row,uric,downs,sex
,Float64,Int64,Int64
1,5.84,0,0
2,6.3,0,0
3,6.95,0,0
4,5.92,0,0
5,7.94,0,0


### Creating Factors

::: callout-note
First we will *transform* our categorical variables from `Int64` to `String` with `@transform!`.

  - For variables with only two levels, we are using an: `if/else` clause.
  
:::

In [4]:
@transform!(
  uric_down,
  :downs = :downs == 0 ? "No" : "Yes",
  :sex = :sex == 0 ? "Male" : "Female"
)

uric_down |> head

Row,uric,downs,sex
,Float64,String,String
1,5.84,No,Male
2,6.3,No,Male
3,6.95,No,Male
4,5.92,No,Male
5,7.94,No,Male


::: callout-note
In the second step, we change the `scitype` of the variable from `String` to `Multiclass`, i.e., a categorical variable, with `coerce`.

:::

In [5]:
coerce!(
  uric_down,
  :downs => Multiclass,
  :sex => Multiclass
)

uric_down |> schema

┌───────┬───────────────┬──────────────────────────────────┐
│ names │ scitypes      │ types                            │
├───────┼───────────────┼──────────────────────────────────┤
│ uric  │ Continuous    │ Float64                          │
│ downs │ Multiclass{2} │ CategoricalValue{String, UInt32} │
│ sex   │ Multiclass{2} │ CategoricalValue{String, UInt32} │
└───────┴───────────────┴──────────────────────────────────┘


### Import from CSV

In [6]:
wcgs = DataFrame(CSV.File("data/wcgs.csv"))
wcgs |> schema

┌─────────┬──────────┬───────┐
│ names   │ scitypes │ types │
├─────────┼──────────┼───────┤
│ id      │ Count    │ Int64 │
│ age0    │ Count    │ Int64 │
│ height0 │ Count    │ Int64 │
│ weight0 │ Count    │ Int64 │
│ sbp0    │ Count    │ Int64 │
│ dbp0    │ Count    │ Int64 │
│ chol0   │ Count    │ Int64 │
│ behpat0 │ Count    │ Int64 │
│ ncigs0  │ Count    │ Int64 │
│ dibpat0 │ Count    │ Int64 │
│ chd69   │ Count    │ Int64 │
│ typechd │ Count    │ Int64 │
│ time169 │ Count    │ Int64 │
│ arcus0  │ Count    │ Int64 │
└─────────┴──────────┴───────┘


Dimensions:

In [7]:
wcgs |> size

(3140, 14)

Names of columns (variables):

In [8]:
wcgs |> names

14-element Vector{String}:
 "id"
 "age0"
 "height0"
 "weight0"
 "sbp0"
 "dbp0"
 "chol0"
 "behpat0"
 "ncigs0"
 "dibpat0"
 "chd69"
 "typechd"
 "time169"
 "arcus0"

### Import from `rds`

In [9]:
kfm = read_rds("data/kfm.rds") |> rcopy
kfm |> head

Row,no,dl_milk,sex,weight,ml_suppl,mat_weight,mat_height
,Int64,Float64,Cat…,Float64,Int64,Int64,Int64
1,1,8.42,Boy,5.002,250,65,173
2,4,8.44,Boy,5.128,0,48,158
3,5,8.41,Boy,5.445,40,62,160
4,10,9.65,Boy,5.106,60,55,162
5,12,6.44,Boy,5.196,240,58,170


### Changing names of variables

In [10]:
wcgs = DataFrames.rename!(
  wcgs,
  :age0 => :age,
  :height0 => :height,
  :weight0 => :weight,
  :sbp0 => :sbp,
  :dbp0 => :dbp,
  :chol0 => :chol,
  :behpat0 => :beh_pat,
  :ncigs0 => :ncigs,
  :dibpat0 => :dib_pat,
  :chd69 => :chd,
  :typechd => :type_chd,
  :time169 => :time,
  :arcus0 => :arcus
);

In [11]:
wcgs[1:5, 2:6]

Row,age,height,weight,sbp,dbp
,Int64,Int64,Int64,Int64,Int64
1,49,73,150,110,76
2,42,70,160,154,84
3,42,69,160,110,78
4,41,68,152,124,78
5,59,70,150,144,86


Let's define factors for the `wcgs` dataset.

In [12]:
@transform!(
  wcgs,
  :chd = :chd == 0 ? "No CHD" : "CHD",
  :arcus = :arcus == 0 ? "Absent" : "Present",
  :dib_pat = :dib_pat == 0 ? "B" : "A"
)
wcgs.beh_pat = recode(
  wcgs.beh_pat, 
  1 => "A1", 2 => "A2",
  3 => "B1", 4 => "B2"
);
wcgs.type_chd = recode(
  wcgs.type_chd, 
  0 => "No CHD", 1 => "MI or SD",
  2 => "Angina", 3 => "Silent MI"
);

In [13]:
coerce!(
  wcgs,
  :chd => Multiclass,
  :arcus => Multiclass,
  :beh_pat => Multiclass,
  :dib_pat => Multiclass,
  :type_chd => Multiclass
);

In [14]:
@select(wcgs, :chd, :arcus, :beh_pat, :dib_pat, :type_chd) |>
schema

┌──────────┬───────────────┬──────────────────────────────────┐
│ names    │ scitypes      │ types                            │
├──────────┼───────────────┼──────────────────────────────────┤
│ chd      │ Multiclass{2} │ CategoricalValue{String, UInt32} │
│ arcus    │ Multiclass{2} │ CategoricalValue{String, UInt32} │
│ beh_pat  │ Multiclass{4} │ CategoricalValue{String, UInt32} │
│ dib_pat  │ Multiclass{2} │ CategoricalValue{String, UInt32} │
│ type_chd │ Multiclass{4} │ CategoricalValue{String, UInt32} │
└──────────┴───────────────┴──────────────────────────────────┘


In [15]:
levels(wcgs.chd)

2-element Vector{String}:
 "CHD"
 "No CHD"

### Transforming to a binary variable

One of our variables is a count and stores the number of smoked cigarettes/day. We can define a new variable `:smoker` in which, everyone who smokes one or more cigarette/day will be a smoker. One of the easiest ways to create binary variables is to use a conditional statement.

In [16]:
@transform!(
  wcgs,
  :smoker = :ncigs == 0 ? "Non-Smoker" : "Smoker"
)
coerce!(wcgs, :smoker => Multiclass)
tabulate(wcgs, :chd, :smoker)

    |        |       smoker        |       
    |        | Non-Smoker | Smoker | Total 
-------------------------------------------
chd |    CHD |         97 |    158 |   255 
    | No CHD |       1549 |   1336 |  2885 
-------------------------------------------
    |  Total |       1646 |   1494 |  3140 


### Simple numeric transformations

We also, prefer units in the metric system. We will convert from inches to centimetres and from pounds to kg.

In [17]:
@transform!(
  wcgs,
  :height = :height * 2.54,
  :weight = :weight * 0.4536
);

### Centring

We can centre a variable by removings its mean. Let’s centre `:height` in `wcgs` as an example.

In [18]:
estat(
	@select(
		@transform(wcgs, :height_cent = @bycol :height .- mean(:height)),
		:height, :height_cent
	)
)

Row,Variable,n,Median,Mean,SD,CV
,Symbol,Int64,Float64,Float64,Float64,Float64
1,height,3140,177.8,177.24,6.404,0.036
2,height_cent,3140,0.56,0.0,6.404,5.20042e13


## Indexing and subsets

Let’s said that we are only interested in subjects who are smokers. If that is the case, we can create a new data frame using the `subset` function from `DataFrameMacros`.

In [19]:
smokers = @subset(wcgs, :smoker == "Smoker")
smokers[1:5, 2:6]

Row,age,height,weight,sbp,dbp
,Int64,Float64,Float64,Int64,Int64
1,49,185.42,68.04,110,76
2,42,177.8,72.576,154,84
3,41,172.72,68.9472,124,78
4,59,177.8,68.04,144,86
5,43,182.88,86.184,146,76


Let's check for the number of observations:

In [20]:
smokers |> nrow

1494

We can access one of those columns easily using `.colname`, which returns a vector that we can access like any Julia vector:

In [21]:
wcgs.chd[1:5]

5-element CategoricalArray{String,1,UInt32}:
 "No CHD"
 "No CHD"
 "No CHD"
 "No CHD"
 "CHD"

In [22]:
@select(wcgs, :ncigs, :smoker) |> head

Row,ncigs,smoker
,Int64,Cat…
1,25,Smoker
2,20,Smoker
3,0,Non-Smoker
4,20,Smoker
5,20,Smoker


:::{.callout-tip}
For negative selection, we use `Not`.
:::

In [23]:
@select(smokers, {Not([:id, :type_chd, :ncigs, :beh_pat])}) |> size

(1494, 11)

:::{.callout-note}
In the case of negative indexing, a list of variables is given in vector format; the equivalent to `c`, the concatenate function in `R`.

:::

:::{.callout-important}
`DataFrameMacros` uses the `{}` expression to include several columns. This is also needed for negative indexing when using `@select`.

:::

In [24]:
@select(kfm, {Not(:no)}) |> head

Row,dl_milk,sex,weight,ml_suppl,mat_weight,mat_height
,Float64,Cat…,Float64,Int64,Int64,Int64
1,8.42,Boy,5.002,250,65,173
2,8.44,Boy,5.128,0,48,158
3,8.41,Boy,5.445,40,62,160
4,9.65,Boy,5.106,60,55,162
5,6.44,Boy,5.196,240,58,170


In [25]:
@select(smokers, {Not(:id, :type_chd, :ncigs, :beh_pat, :smoker)}) |>
head

Row,age,height,weight,sbp,dbp,chol,dib_pat,chd,time,arcus
,Int64,Float64,Float64,Int64,Int64,Int64,Cat…,Cat…,Int64,Cat…
1,49,185.42,68.04,110,76,225,A,No CHD,1664,Absent
2,42,177.8,72.576,154,84,177,A,No CHD,3071,Present
3,41,172.72,68.9472,124,78,132,B,No CHD,3064,Absent
4,59,177.8,68.04,144,86,255,B,CHD,1885,Present
5,43,182.88,86.184,146,76,149,B,No CHD,3064,Absent


::: callout-tip
## Example

Suppose we want to know the number of smokers who weight 100 kg or more.
:::

In [26]:
@subset(smokers, :weight >= 100) |>
nrow

20

::: callout-note
For obtaining the same result, but from the original dataset, we would need to use two conditionals. We use the syntax `&&` for **AND** and the syntax `||` for **OR** We can also use `,` for **AND**.
:::

In [27]:
@subset(wcgs, :smoker == "Smoker", :weight >= 100) |> nrow

20

::: callout-caution
## Exercise

Determine the number of smokers who have behavioural pattern `A2` and had either angina or silent myocardial infarction. You will need to write the OR conditionals between parenthesis.
:::

In [28]:
#| code-fold: true
@subset(smokers, :beh_pat == "A2", (:type_chd == "Angina" || :type_chd == "Silent MI")) |>
nrow

38

::: callout-tip
It's also possible to answer the previous question using `in`, as shown bellow.
:::

In [29]:
@subset(smokers, :beh_pat == "A2", :type_chd in ("Angina", "Silent MI")) |> nrow

38

::: callout-tip
## Example

Let’s say we want to know all the variable values associated with the subject who has the maximum weight in the `wcgs` dataset. With `findmax` we get both the value and the observation number.
:::

In [30]:
wcgs.weight |> findmax

(145.152, 1090)

In [31]:
wcgs[1090, :]

Row,id,age,height,weight,sbp,dbp,chol,beh_pat,ncigs,dib_pat,chd,type_chd,time,arcus,smoker
,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Cat…,Int64,Cat…,Cat…,Cat…,Int64,Cat…,Cat…
1090,10078,43,193.04,145.152,166,102,188,B1,0,B,CHD,Angina,1795,Absent,Non-Smoker


In [32]:
wcgs[findmax(wcgs.weight)[2], :]

Row,id,age,height,weight,sbp,dbp,chol,beh_pat,ncigs,dib_pat,chd,type_chd,time,arcus,smoker
,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Cat…,Int64,Cat…,Cat…,Cat…,Int64,Cat…,Cat…
1090,10078,43,193.04,145.152,166,102,188,B1,0,B,CHD,Angina,1795,Absent,Non-Smoker


## Matrices and Missing Values

### Converting the data

If we want to get the content of the dataframe as one big matrix, use `Matrix`:

In [33]:
kfm_mat = Matrix(
  @select(kfm, {Not(1, 3, 5)})
);

In [34]:
@show(size(kfm));
@show(size(kfm_mat));

size(kfm) = (50, 7)
size(kfm_mat) = (50, 4)


In [35]:
kfm_mat[1:5, :]

5×4 Matrix{Float64}:
 8.42  5.002  65.0  173.0
 8.44  5.128  48.0  158.0
 8.41  5.445  62.0  160.0
 9.65  5.106  55.0  162.0
 6.44  5.196  58.0  170.0

### Missing values

In [36]:
mao = dataset("gap", "mao")
mao |> schema

┌──────────┬─────────────────────────┬─────────────────────────┐
│ names    │ scitypes                │ types                   │
├──────────┼─────────────────────────┼─────────────────────────┤
│ ID       │ Textual                 │ String15                │
│ Type     │ Count                   │ Int64                   │
│ Gender   │ Count                   │ Int64                   │
│ Age      │ Union{Missing, Count}   │ Union{Missing, Int64}   │
│ AAO      │ Union{Missing, Count}   │ Union{Missing, Int64}   │
│ AAD      │ Union{Missing, Count}   │ Union{Missing, Int64}   │
│ UPDRS    │ Union{Missing, Count}   │ Union{Missing, Int64}   │
│ MAOAI2   │ Textual                 │ String7                 │
│ AI2Code  │ Textual                 │ String7                 │
│ MAOBI2   │ Textual                 │ String7                 │
│ BI2Code  │ Textual                 │ String3                 │
│ GTBEX3   │ Textual                 │ String7                 │
│ BEX3Code │ Textual     

In [37]:
mao[1:7, 2:6]

Row,Type,Gender,Age,AAO,AAD
,Int64,Int64,Int64?,Int64?,Int64?
1,0,0,69,missing,missing
2,0,0,missing,missing,missing
3,0,0,missing,missing,missing
4,0,0,missing,missing,missing
5,0,1,missing,missing,missing
6,0,1,missing,missing,missing
7,0,1,missing,missing,missing


In [38]:
describe(mao, :nmissing)

Row,variable,nmissing
,Symbol,Int64
1,ID,0
2,Type,0
3,Gender,0
4,Age,188
5,AAO,296
6,AAD,295
7,UPDRS,301
8,MAOAI2,0
9,AI2Code,0


Lots of missing values $\ldots$ If we wanted to compute simple functions on columns, they may just return missing:

In [39]:
std(mao.Age)

missing

Some functions remove missing values under a `@chain` pipe:

In [40]:
describe(
  @select(mao, :Age),
  :min, :max, :mean, :median, :std, rel_dis => :cv
)

Row,variable,min,max,mean,median,std,cv
,Symbol,Int64,Int64,Float64,Float64,Float64,Float64
1,Age,26,91,73.5395,77.0,11.5517,0.157082


We can use `skipmissing`, or `dropmissing` to counter this easily:

In [41]:
std(skipmissing(mao.Age)) |> r3

11.552

## Group Manipulations

### Split-Apply-Combine

In [42]:
iris = rcopy(R"datasets::iris")
iris |> schema

┌──────────────┬───────────────┬──────────────────────────────────┐
│ names        │ scitypes      │ types                            │
├──────────────┼───────────────┼──────────────────────────────────┤
│ Sepal_Length │ Continuous    │ Float64                          │
│ Sepal_Width  │ Continuous    │ Float64                          │
│ Petal_Length │ Continuous    │ Float64                          │
│ Petal_Width  │ Continuous    │ Float64                          │
│ Species      │ Multiclass{3} │ CategoricalValue{String, UInt32} │
└──────────────┴───────────────┴──────────────────────────────────┘


The `groupby` function allows to form sub-dataframes corresponding to groups of rows. This can be very convenient to run specific analyses for specific groups without copying the data.

The basic usage is `groupby(df, cols)` where `cols` specifies one or several columns to use for the grouping.

Consider a simple example: in `iris` there is a `Species` column with 3 species:

In [43]:
@select(iris, :Species) |> unique

Row,Species
,Cat…
1,setosa
2,versicolor
3,virginica


We can construct views for each of these:

In [44]:
gdf = @groupby(iris, :Species);

In [45]:
subdf_setosa = gdf[1]
describe(subdf_setosa, :mean, :median, :std, rel_dis => :cv)

Row,variable,mean,median,std,cv
,Symbol,Union…,Union…,Union…,Union…
1,Sepal_Length,5.006,5.0,0.35249,0.0704134
2,Sepal_Width,3.428,3.4,0.379064,0.110579
3,Petal_Length,1.462,1.5,0.173664,0.118785
4,Petal_Width,0.246,0.2,0.105386,0.428397
5,Species,,,,


Or using pipes, without the need of creating subsets:

In [46]:
estat(
  @select(
    @subset(iris, :Species .== "setosa"),
    {Not(:Species)}
  )
)

Row,Variable,n,Median,Mean,SD,CV
,Symbol,Int64,Float64,Float64,Float64,Float64
1,Sepal_Length,50,5.0,5.006,0.352,0.07
2,Sepal_Width,50,3.4,3.428,0.379,0.111
3,Petal_Length,50,1.5,1.462,0.174,0.119
4,Petal_Width,50,0.2,0.246,0.105,0.428


:::{.callout-tip}
We can make transformations during the selection process, even to multiple columns if we use the `@select` macro.
:::

Let's change the scale, for all continuous variables, as an example.

In [47]:
estat(
  @select(
    @subset(iris, :Species .== "setosa"),
    {1:4} * 10
  )
)

Row,Variable,n,Median,Mean,SD,CV
,Symbol,Int64,Float64,Float64,Float64,Float64
1,Sepal_Length_function,50,50.0,50.06,3.525,0.07
2,Sepal_Width_function,50,34.0,34.28,3.791,0.111
3,Petal_Length_function,50,15.0,14.62,1.737,0.119
4,Petal_Width_function,50,2.0,2.46,1.054,0.428


We can use `combine` and `groupby` to estimate the 95% CIs of blood cholesterol from subjects in the `wcgs` data set. In this example, the function `cis` reports the expected value (mean) in the column `outcome`.

In [48]:
combine(groupby(wcgs |> dropmissing, [:chd, :beh_pat]), :chol => cis => AsTable)

Row,chd,beh_pat,outcome,lower,upper
,Cat…,Cat…,Float64,Float64,Float64
1,CHD,A1,249.8,235.805,263.795
2,CHD,A2,248.701,239.974,257.428
3,CHD,B1,250.55,239.221,261.879
4,CHD,B2,258.056,237.296,278.815
5,No CHD,A1,230.318,224.964,235.672
6,No CHD,A2,225.466,223.02,227.912
7,No CHD,B1,222.887,220.464,225.31
8,No CHD,B2,220.488,216.023,224.953
